In [1]:
# Telling matplotlib to work in the program and not as an external window
%matplotlib inline                  
from nptdms import TdmsFile           # Handling TDMS files
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from numpy import *
import matplotlib.cm as cm
import time
from scipy.optimize import curve_fit
import scipy
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns; import pandas as pd
import warnings
from scipy.optimize import OptimizeWarning
import os, sys
from copy import deepcopy            # To completely copy lists and not references
from pandas import DataFrame, Series  # for convenience
import cv2                           # OpenCV for video and image procedures
                                   # generating a folder or deleting files, etc.
import multiprocessing as mp         # This is for preventing large videos to take too much internal memory
import shutil                        # Deleting folders
plt.rcParams.update({'font.size': 14})
import matplotlib.cm as cm
import statistics as stats

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def calcvelocity(rrange,sigma,vdiff,A,vmin):
    sigmaterm = 2*sigma**2
    distances = np.linspace(0,rrange,100)
    I = A*np.exp(-distances**2/sigmaterm)
    return distances, vdiff*np.exp(-2*I/A)+vmin

def analyseTDMS(file):
    # Print properties and channels of TDMS file
    tdms_file = TdmsFile(file)
    print('Properties (Root):')
    for name, value in tdms_file.object().properties.items():
        print(2*' ' + "{0}: {1}".format(name, value))
    for group in tdms_file.groups():
        print('\'' + group + '\'')
        print(2*' ' + 'Properties (' + '\'' + group + '\'' ')')
        for name, value in tdms_file.object(group).properties.items():
            print('')
        for channel in tdms_file.group_channels(group):
            print(2*' ' + channel.channel)
            
def disk_line_picking(s,Rin):
    return 4*s/(np.pi*Rin**2)*np.arccos(s/(2*Rin))-2*s**2/(np.pi*Rin**3)*np.sqrt(1-s**2/(4*Rin**2))

In [2]:
start = 12
stop = 24

folder = "/Users/alex/Documents/Measurements/201216 - Alex 2 particles"
savefolder = "."
maxpower = 2.4
num = stop - start
pxtomum=0.0533
binning = 1
frametransfer = False
flashtime = 0.01
verticalshift = 1.9176e-6
dtcamtoPC = 1e-3
dtPCtoADwin = 1e-3

R = 2.19e-6/2
diameter = R*2*1e6
T = 273.15+22
kB = 1.38064852e-23
eta=2.414e-5*10**(247.8/(T-140))
D_0 = kB*T/(6*np.pi*eta*R)
D_0 = 3.85e-14
amplitude = 1
steps = 10#20#13

plotCoM = True#False
plotvelocityvariation = False
plothisto = True#False
plotvelocityinout = True
plotcollision = True
plottrajectory = True

In [3]:
totaltime = 0
for m in range(start,stop+1):
#for m in range(6,stop+1):
#for m in range(start,start+1):
    start_time = time.time() 
    filenumber = m
    if filenumber < 10:
            tdms_file = TdmsFile(folder+"/Test_00"+str(filenumber)+"_module.tdms")
            #analyseTDMS(folder+"/Test_00"+str(number)+"_module.tdms")
    else:
        if filenumber < 100:
            tdms_file = TdmsFile(folder+"/Test_0"+str(filenumber)+"_module.tdms")
            #analyseTDMS(folder+"/Test_0"+str(number)+"_module.tdms")
        else:
            tdms_file = TdmsFile(folder+"/Test_"+str(filenumber)+"_module.tdms")
            #analyseTDMS(folder+"/Test_"+str(number)+"_module.tdms")
    print("--- %s seconds ---" % round((time.time() - start_time),2))

    #Xc = tdms_file.channel_data('Data','Xc')
    Xc = tdms_file['Data']['Xc']
    Yc = tdms_file['Data']['Yc']
    velocitytheo = tdms_file['Data']['Velocity']
    intensitytheo = tdms_file['Data']['Intensity']
    angletheo = tdms_file['Data']['Angle']
    xolaser = tdms_file['Data']['Xh']
    yolaser = tdms_file['Data']['Yh']
    Nfound = tdms_file['Data']['N']
    Nlaser = tdms_file['Data']['NLaser']
    delaywave = tdms_file['Data']['Delays']
    iterationtime = tdms_file['Data']['Iterationtime']
    programtime = tdms_file['Data']['Programtime']
    phase = tdms_file['Data']['Phase']
    p = tdms_file.object().properties
    delay = int(p['Delay'])
    noise = int(p['Sigmanoise'])
    boundaryradius = p['Boundary']
    offset = p['Offset']
    sigmavelo = p['Sigmavelo']
    vmin = p['Vmin']
    vmax = p['Vmax']
    dt = p['CycleTime']
    videosize = p['Xsize']
    vfactor = vmax - vmin
    firstlength = len(Nfound)
    maxlength = 0
    for i in range(firstlength):
        if phase[i] == 0: maxlength += 1
    timewave=np.linspace(0,maxlength*dt,maxlength)
    dtprogram = np.mean(programtime)*1e-3#6.3e-3
    dtexphalf = flashtime/2
    dtframetransfer = videosize/binning * verticalshift
    dtreadout = (1032/27e6+verticalshift)*videosize/binning
    mindelay = dtexphalf + dtframetransfer + dtreadout + dtcamtoPC + dtprogram + dtPCtoADwin
    delaydt = mindelay +(delay-1)*dt
    savefolder = 'Noise '+str(noise)
    savefolderplus = savefolder
    plotnum = str(noise) + ' - ' + str(delay)
    #------------------------------------------
    number = int(stats.mode(Nfound))     #!!!!!!!!!!!!!!! CHANGE THAT NUMBER
    pnumber = number
    maxnumber = int(max(Nfound))
    particles = np.zeros((maxlength,number,9))
    lastone = np.zeros((number,2))
    pre = np.zeros((maxlength,maxnumber,9))
    test = np.zeros((int(Nfound[0]),2))
    for i in range(int(Nfound[0])):             #Sort the particles in the first frame according to there 
        test[i,0] = i                           #distance to the center, particles outside the boundary are not important
        test[i,1] = np.sqrt(pow(Xc[i]-videosize/2,2)+pow(Yc[i]-videosize/2,2))
    check = sorted(test, key=lambda test: test[1])
    order = np.zeros(int(Nfound[0]))
    for i in range(int(Nfound[0])):
        test2 = check[i]
        order[i] = test2[0]  
    count = 0
    for j in range(int(Nfound[0])):
        pre[0,j,0] = Xc[int(order[j])]
        pre[0,j,1] = Yc[int(order[j])]
        pre[0,j,2] = velocitytheo[int(order[j])]
        pre[0,j,3] = angletheo[int(order[j])]  
        pre[0,j,4] = xolaser[int(order[j])]
        pre[0,j,5] = yolaser[int(order[j])]
        pre[0,j,6] = intensitytheo[int(order[j])] 
    count = int(Nfound[0])
    for i in range(1,maxlength):               #Write all position in a first wave
        for j in range(int(Nfound[i])):
            pre[i,j,0] = Xc[count]
            pre[i,j,1] = Yc[count]
            pre[i,j,2] = velocitytheo[count]
            pre[i,j,3] = angletheo[count]
            pre[i,j,4] = xolaser[count]
            pre[i,j,5] = yolaser[count]
            pre[i,j,6] = intensitytheo[count]
            count += 1

    sumup = Nfound[0]                         #sumup - if the first frames are without a particle, they willbe cut off
    if Nfound[0] != 0: starttraj = 0
    particles[0,:,:] = pre[0,:number,:]
    diff = 0
    for i in range(1,maxlength):
        if sumup != 0:
            check = False; occupied = np.zeros((int(Nfound[i]),2))
            for p in range(2):                    #first check those particles which where found in the frame before
                for j in range(pnumber):
                    if p == 0:
                        if particles[i-1,j,0] != 0: check = True
                    else:
                        if particles[i-1,j,0] == 0: check = True          
                    if check == True:         
                        sumup += Nfound[i]
                        if particles[i-1,j,0] != 0:    #remember how long the particle is lost
                            lastone[j,0] = i-1
                            lastone[j,1] = 0
                        else: lastone[j,1] += 1
                        xpre = particles[int(lastone[j,0]),j,0]
                        ypre = particles[int(lastone[j,0]),j,1]
                        mindist = 100; minnum = 666
                        for k in range(int(Nfound[i])):     #Sort the trajectories be comparing the distance between the positions
                            if occupied[k,0] == 0:
                                dist = np.sqrt(pow(xpre-pre[i,k,0],2)+pow(ypre-pre[i,k,1],2))
                                if dist < mindist:
                                    mindist = dist
                                    minnum = k
                                    #print('i = ',i,' - j = ',j,' - mindist = ',round(mindist,1),' - minnum = ',minnum)
                        if minnum != 666:              #Write the shortest one
                            particles[i,j,:4] = pre[i,minnum,:4]
                            particles[i,j,6] = pre[i,minnum,6]
                            occupied[minnum,0] = 1
                        mindist = 0.2; minnum = 666     #sort the laser position
                        for k in range(int(Nfound[i])):
                            if particles[i,j,0] != 0:
                                if occupied[k,1] == 0:
                                    dist = np.sqrt(pow(particles[i-diff,j,0]-pre[i,k,4],2)+pow(particles[i-diff,j,1]-pre[i,k,5],2))
                                    #print('i = ',i,' - j = ',j,' - k = ',k,' - dist = ',round(dist,1))
                                    if abs(dist - offset) < mindist:
                                        mindist = dist
                                        minnum = k
                        if minnum != 666:              #Write the shortest one
                            particles[i,j,4:6] = pre[i,minnum,4:6]
                            occupied[minnum,1] = 1                           
        else:
            if Nfound[i] != 0:
                starttraj = i
                sumup += Nfound[i]
                for j in range(int(Nfound[i])): particles[i,j,:] = pre[i,j,:]
        #print('i = ',i,' - Nfound = ',Nfound[i],' - x[i,0] = ',int(particles[i,0,0]))

    for i in range(1,maxlength):
        for j in range(number):
            if particles[i,j,0] == 0:
                particles[i,j,0] = np.nan
                particles[i,j,1] = np.nan  
            if particles[i,j,4] == 0: 
                particles[i,j,4] = np.nan
                particles[i,j,5] = np.nan
    particles[:,:,0:2] = (particles[:,:,0:2]-videosize/2)*pxtomum
    particles[:,:,4:6] = (particles[:,:,4:6]-videosize/2)*pxtomum
    save = particles.copy()
    cutlastpoints = False
    trajlength = np.zeros(number)
    for j in range(number):
        lastfound = 0; out = True
        for i in range(starttraj+1,maxlength):
            if out == True:
                if particles[i,j,0] != particles[i,j,0]:
                    lastfound = int(i-1)
                    count = 1; check = True
                    while check == True:
                        if particles[lastfound+count,j,0] == particles[lastfound+count,j,0]:
                            check = False
                        else: count +=1
                        if count == 150:
                            print('Failed at j = ',j,' - i = ',i)
                            check = False
                            trajlength[j] = lastfound
                            particles[lastfound:,j,0] = np.nan
                            particles[lastfound:,j,1] = np.nan
                            out = False
                    if out == True:
                        for k in range(1,count+1):
                            particles[lastfound+k,j,0] = particles[lastfound,j,0]+(k/(count+1))*(particles[lastfound+count,j,0]-particles[lastfound,j,0])
                            particles[lastfound+k,j,1] = particles[lastfound,j,1]+(k/(count+1))*(particles[lastfound+count,j,1]-particles[lastfound,j,1])
                            particles[lastfound+k,j,2] = particles[lastfound,j,2]+(k/(count+1))*(particles[lastfound+count,j,2]-particles[lastfound,j,2])
                            particles[lastfound+k,j,3] = particles[lastfound,j,3]+(k/(count+1))*(particles[lastfound+count,j,3]-particles[lastfound,j,3])
                            particles[lastfound+k,j,6] = particles[lastfound,j,6]+(k/(count+1))*(particles[lastfound+count,j,6]-particles[lastfound,j,6])
                if i == firstlength-1: trajlength[j] = i
    particlessave = particles.copy()
    if starttraj != 0:
        maxlength = int(maxlength-starttraj)
        particles = np.zeros((maxlength,number,6))
        particles = particlessave[starttraj:,:,:]
    if cutlastpoints == True:
        particlessave = particles.copy()
        delete = firstlength - trajlength[0]
        maxlength = int(maxlength-delete)
        particles = np.zeros((maxlength,number,6))
        particles = particlessave[:maxlength,:,:]
    timewave=np.linspace(0,maxlength*dt,maxlength)
    
    for i in range(maxlength):
        for j in range(pnumber):
            if particles[i,j,6] > 1: particles[i,j,6] = 1

    textwave = ['x','y','Laserx','Lasery','Angle','General']
    for i in range(6):
        string_temp = savefolderplus + '/Data'+str(textwave[i])+'/'
        if not os.path.exists(string_temp): os.makedirs(string_temp)
        if i == 0: np.savetxt(string_temp + 'Datax'+str(filenumber)+'.txt', np.transpose([particles[:,0,0],particles[:,1,0]]),fmt='%1.6f')
        if i == 1: np.savetxt(string_temp + 'Datay'+str(filenumber)+'.txt', np.transpose([particles[:,0,1],particles[:,1,1]]),fmt='%1.6f')
        if i == 2: np.savetxt(string_temp + 'DataLaserx'+str(filenumber)+'.txt', np.transpose([particles[:,0,4],particles[:,1,4]]),fmt='%1.6f')
        if i == 3: np.savetxt(string_temp + 'DataLasery'+str(filenumber)+'.txt', np.transpose([particles[:,0,4],particles[:,1,4]]),fmt='%1.6f')
        if i == 4: np.savetxt(string_temp + 'DataAngle'+str(filenumber)+'.txt', np.transpose([particles[:,0,3],particles[:,1,3]]),fmt='%1.6f')
        if not os.path.exists('./DataGeneral'): os.makedirs('./DataGeneral')
        if i == 5: np.savetxt('./DataGeneral/DataGeneral'+str(filenumber)+'.txt', np.transpose([dt,pxtomum,offset,videosize,boundaryradius,maxpower,sigmavelo,vmin,vmax,amplitude]),fmt='%1.6f')
#---------------------------------------------------              
    directionvector = np.zeros((maxlength,2))
    movingvector = np.zeros((maxlength,2))
    velocity = np.zeros((maxlength,number));stepsize = np.zeros(maxlength)
    velocitywithdiffusion=np.zeros((maxlength,number))
    velocity2 = np.zeros((maxlength,number,2))
    distancetocenter = np.zeros((maxlength,number))
    for j in range(number):
        directionvector[:,0] = np.cos(particles[:,j,2]*np.pi/180)
        directionvector[:,1] = np.sin(particles[:,j,2]*np.pi/180)
        for i in range(1,maxlength):                           #Calc projection of the velocity in the required direction
            distancetocenter[i,j] = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            movingvector[i,0] = particles[i,j,0]-particles[i-1,j,0]
            movingvector[i,1] = particles[i,j,1]-particles[i-1,j,1]
            stepsize[i] = abs(np.dot(directionvector[i,:], movingvector[i,:]))
            velocity[i,j] = stepsize[i]/dt
            velocitywithdiffusion[i,j] = np.sqrt(pow(movingvector[i,0],2)+pow(movingvector[i,1],2))/dt 
            distance = np.sqrt(pow(particles[i,j,0],2)+pow(particles[i,j,1],2))
            if distancetocenter[i,j] < boundaryradius*pxtomum:
                velocity2[i,j,0] = velocity[i,j]
                velocity2[i,j,1] = velocitywithdiffusion[i,j]
    for j in range(number):
        for i in range(maxlength):
            if velocity2[i,j,0] == 0: velocity2[i,j,:]=np.nan

    wave = velocity2[:,:,0]
    binwave=np.linspace(np.amin(wave[~np.isnan(wave)]),min(np.amax(wave[~np.isnan(wave)]),10),20)
    string_temp = savefolder + '/PlotVelocityHisto/'
    if not os.path.exists(string_temp): os.makedirs(string_temp) 
    fig=plt.figure(1, figsize = (7,4) )
    plt.subplot(1,1,1)
    for i in range(number):
        wave = velocity2[:,i,0]    
        plt.hist(wave[~np.isnan(wave)], bins=binwave,alpha = 0.2)
        plt.axvline(x=np.nanmean(wave),linewidth=0.5, color = 'k',linestyle='--')
    plt.xlabel('Velocity [$\mu m/s$]')
    plt.ylabel('accurance')
    #plt.text(3,0,'mean step size 1: {:.2f} µm/s'.format(np.mean(velocity)),fontsize=12)
    plt.legend()
    plt.tight_layout() 
    plt.savefig(string_temp+'Velocities'+str(plotnum)+'.pdf', format='pdf')
    plt.close(fig)#plt.show()   
    #--------------------------------------------------------
    distcheck = np.zeros((maxlength,number))
    sigma = 2 * sigmavelo**2
    for i in range(maxlength):
        for j in range(number):
            particles[i,j,7] = 0
            for k in range(number):
                if j != k:
                    dx = particles[i,j,0]-particles[i,k,0]
                    dy = particles[i,j,1]-particles[i,k,1]
                    dist = np.sqrt(dx*dx+dy*dy)/pxtomum
                    distcheck[i,j] = dist
                    particles[i,j,7] += 2*np.exp(-dist*dist/sigma)
            particles[i,j,7] /= number
            if particles[i,j,7] > 1: particles[i,j,7] = 1

    comlocation = np.zeros((maxlength,5))
    for i in range(maxlength):
        if prod(particles[i,:,0]) != 0:
            comlocation[i,0] = np.mean(particles[i,:,0])
            comlocation[i,1] = np.mean(particles[i,:,1])
    for i in range(maxlength):
        if comlocation[i,0] == 0:
            pre = i-1
            check = False; checknumber = i+1
            while check == False:
                if checknumber < maxlength:
                    if comlocation[checknumber,0] != 0:
                        post = checknumber
                        check = True
                    else:
                        checknumber += 1
                else:
                    check = True
            comlocation[i,:] = (comlocation[pre,:] + comlocation[post,:])/2

    for i in range(maxlength):
        for j in range(number):
            particles[i,j,8] = np.sqrt(pow(particles[i,j,0]-comlocation[i,0],2)+pow(particles[i,j,1]-comlocation[i,1],2))
        comlocation[i,2] = np.mean(particles[i,:,8])
        comlocation[i,3] = np.mean(particles[i,:,7])
        comlocation[i,4] = np.mean(particles[i,:,2])
    meandisttocom = np.nanmean(comlocation[:,2])
    meanintensity = np.nanmean(comlocation[:,3])
    
    test = np.zeros(maxlength)
    for i in range(maxlength):
        count = 0
        for j in range(number):
            if particles[i,j,7] > 0.4: count += 1
        if count >= 2: test[i] = 1
    percentagecluster = sum(test)/maxlength
    #------------------------------------------------------------
    connections = int(number*(number-1)/2)
    distances = np.zeros((maxlength,connections))
    calclength = np.zeros(connections)
    boundmum = boundaryradius * pxtomum
    count = 0; totallength = 0
    for j in range(number):
        for k in range(j+1,number):
            calclength[count] = int(min(trajlength[j],trajlength[k]))
            for i in range(int(calclength[count])):
                if distancetocenter[i,j] < boundmum and distancetocenter[i,k] < boundmum:
                    distances[i,count] = np.sqrt(pow(particles[i,j,0]-particles[i,k,0],2)+pow(particles[i,j,1]-particles[i,k,1],2))
            totallength += calclength[count]
            count += 1
    alldistances = np.zeros(int(totallength))
    calcstart = 0; calcend = 0
    for i in range(connections):
        calcend += calclength[i] 
        alldistances[int(calcstart):int(calcend)] = distances[:int(calclength[i]),i]
        calcstart += calclength[i]
    for i in range(int(totallength)):
        if alldistances[i] == 0: alldistances[i] = np.nan
    alldistances = alldistances[~np.isnan(alldistances)]      
    D_arena = 2*boundmum; D = 2*R*1e6
    #theoradius2 = np.linspace(D,D_arena,10000)
    #dr = theoradius2[1] - theoradius2[0]
    #mean2 = sum(theoradius2*disk_line_picking(theoradius2,D_arena/2)*dr)
    meansim = 14.712#np.mean(allsimdistances)
    meanexp = np.mean(alldistances)

    string_temp = savefolder + '/PlotParticledistance/'
    if not os.path.exists(string_temp): os.makedirs(string_temp) 
    fig=plt.figure(1, figsize = (7,5) )
    ax = plt.subplot(1,1,1)
    bins = np.linspace(min(alldistances),D_arena,30)
    #plt.hist(allsimdistances,bins = bins,density = 1,alpha=0.5,label='simulation')
    plt.hist(alldistances,bins = bins,density = 1,color='red',alpha=0.5,label='experiment')
    plt.axvline(x=2*boundmum,linewidth=0.5, color = 'k',linestyle='--')
    theoradius = np.linspace(0,2*boundmum,1000)
    theory = disk_line_picking(theoradius,boundmum)
    plt.plot(theoradius,theory,color='k',label='theory')
    plt.xlabel('Particle distance [µm]')
    plt.ylabel('Density')
    plt.xlim([0,D_arena])
    plt.ylim([0,1.5*max(theory)])
    plt.legend()
    plt.tight_layout() 
    plt.savefig(string_temp+'Particle Distance distribution'+str(filenumber)+'.pdf', format='pdf')
    plt.close(fig)#plt.show() 

    if plotCoM == True:
        string_temp = savefolder + '/PlotVelocityvsCoM-intensity/'
        if not os.path.exists(string_temp): os.makedirs(string_temp) 
        fig=plt.figure(1, figsize = (14,10) )
        ax = plt.subplot(2,2,1)
        for i in range(number):
            plt.scatter(particles[:,i,6],particles[:,i,2],s=2,c='grey',alpha=0.1)
        plt.xlabel('Distance to the CoM [µm]')
        plt.ylabel('intended Velocity')
        plt.xlim([0,np.amax(particles[:,:,6])])

        ax = plt.subplot(2,2,2)
        for i in range(number):
            plt.scatter(particles[:,i,7],particles[:,i,2],s=2,c='grey',alpha=0.1)
        plt.xlabel('Sensed intensity')
        plt.ylabel('intended Velocity')
        plt.xlim([1,0])

        smoothvalue = 20
        smoothit = np.zeros((maxlength,number+1))
        for i in range(number): smoothit[:,i] = smooth(velocity[:,i],smoothvalue)
        for i in range(maxlength): smoothit[i,-1] = mean(smoothit[i,:-1])

        ax = plt.subplot(2,2,3)
        #plt.scatter(comlocation[:,2],comlocation[:,4],s=2,c='grey',alpha=0.1)
        plt.scatter(comlocation[:,2],smoothit[:,-1],s=2,c='grey',alpha=0.1)
        plt.xlabel('Average Distance to the CoM [µm]')
        plt.ylabel('Average velocity [µm/s]')
        plt.xlim([0,np.amax(particles[:,:,6])])

        ax = plt.subplot(2,2,4)
        #plt.scatter(comlocation[:,3],comlocation[:,4],s=2,c='grey',alpha=0.1)
        plt.scatter(comlocation[:,3],smoothit[:,-1],s=2,c='grey',alpha=0.1)
        plt.xlabel('Average sensed Intensity')
        plt.ylabel('Average velocity [µm/s]')
        plt.xlim([1,0])
        plt.savefig(string_temp+'Velocity vs CoM-intensity'+str(plotnum)+'.pdf', format='pdf')
        plt.savefig(string_temp+'Velocity vs CoM-intensity'+str(filenumber)+'.png', format='png')
        plt.close(fig)#plt.show() 
    #--------------------------------------------------
    if plotvelocityvariation == True:
        laserangle = np.zeros((maxlength,pnumber));diffangle = np.zeros((maxlength,pnumber))
        moveangle = np.zeros((maxlength,pnumber));diffmoveangle = np.zeros((maxlength,pnumber))
        delaylaserangle = 0   #in frames
        delaydirection = 2    #in frames
        angle = particles[:,:,3]
        for j in range(pnumber):
            for i in range(int(trajlength[j])):
                if particles[i,j,4] == particles[i,j,4]:
                    dx = particles[i,j,0]-particles[i,j,4]             #Calc the angle of the laser
                    dy = particles[i,j,1]-particles[i,j,5]
                    laserangle[i,j] = np.arctan(dy/dx)*180/np.pi
                    #laserangle[i] = - laserangle[i]
                    if laserangle[i,j] != laserangle[i,j]: laserangle[i,j] = 90
                    if dx > 0:
                        if dy < 0: laserangle[i,j] += 360
                    else: laserangle[i,j] += 180
                    if laserangle[i,j]>360: laserangle[i,j] -= 360
                    if i >= delaylaserangle:                      #Calc difference between required laser angle and real angle
                        diffangle[i,j] = particles[i-delaylaserangle,j,3] - laserangle[i,j]
                        if abs(diffangle[i,j]) > 180:
                                if diffangle[i,j] > 0: diffangle[i,j] -= 360
                                else: diffangle[i] += 360
                        if abs(diffangle[i,j]) > 40: diffangle[i,j] = 0
                            #print('Error laser? : i = ',i)
        parts = 16
        saveradi = np.zeros((parts,pnumber))   
        meanv = np.zeros(pnumber)
        for k in range(pnumber):
            distlaser = np.zeros(int(trajlength[k]))
            distangle = np.zeros((parts,4))     #0 - angle, 1 - distlaser, 2 - counts, 3 - velocity
            for i in range(parts): distangle[i,0] = (i+1)/parts * 360
            for i in range(int(trajlength[k])-1):                        #Statistic over the laser offset (and the velocity in required direction) vs the required direction
                if particles[i,k,4] == particles[i,k,4]:
                    distlaser[i] = np.sqrt(pow(particles[i,k,4]-particles[i,k,0],2)+pow(particles[i,k,5]-particles[i,k,1],2))
                    if abs(distlaser[i]-offset*pxtomum) > 1.5:
                        #print('To high laser offset, t = ',round(i*dt,2),' s - dist = ',round(abs(distlaser[i]-offset),1),' µm')
                        distlaser[i] = distlaser[i-1]
                    if laserangle[i,k]<distangle[0,0]:
                        distangle[0,1] += distlaser[i]
                        distangle[0,2] += 1
                        distangle[0,3] += velocity[i,k]
                    else:
                        for j in range(1,parts):
                            if laserangle[i,k] > distangle[j-1,0] and laserangle[i,k] < distangle[j,0]:
                                if distancetocenter[i,k] < boundaryradius*pxtomum:
                                    distangle[j,1] += distlaser[i]
                                    distangle[j,2] += 1
                                    distangle[j,3] += velocity[i,k]     
            distangle[:,1] /= distangle[:,2]
            distangle[:,3] /= distangle[:,2]
            timewave = np.linspace(0,maxlength*dt,maxlength,endpoint=False) 
            meanv[k] = np.nanmean(velocity2[:,k,0])#np.nanmean(distangle[:,3])
            saveradi[:,k] = distangle[:,3] - meanv[k]

        fig=plt.figure(1, figsize = (5*pnumber,5) ) 
        for k in range(pnumber):
            theta = np.linspace(0, 2 * np.pi, parts, endpoint=False)
            radii = saveradi[:,k]
            width = 2*np.pi / (parts)
            theta += width/2
            ax = plt.subplot('1'+str(pnumber)+str(k+1), projection='polar')
            ax.set(aspect="equal",title='Particle '+str(k+1)+' - '+str(round(meanv[k],1))+' µm/s')
            bars = ax.bar(theta, radii, width=width, bottom=0.0)
            for r, bar in zip(radii, bars):
                bar.set_facecolor(plt.cm.viridis(r))
                bar.set_alpha(0.5)
            ax.set_rmin(np.amin(saveradi))
            ax.set_rmax(np.amax(saveradi))
        plt.tight_layout()   
        plt.savefig(string_temp+'Velocity angle'+str(plotnum)+'.pdf', format='pdf')
        plt.close(fig)#plt.show() 

    maxvelocity = np.zeros(number)
    for k in range(number):
        value = particles[:,k,2]
        steps2 = 6
        speedhisto= np.zeros((steps2,4))    #0 - angle, 1 - velocity, 2 - counts, 3 - variance
        speedcollect=np.zeros((steps2,maxlength))
        for i in range(steps2): speedhisto[i,0] = (i+1)/steps2
        for i in range(maxlength):                      #Statistics over the velocity in required direction vs the theoretical velocity
            if value[i] < speedhisto[0,0]:
                if velocity[i,k] == velocity[i,k]:
                    speedhisto[0,1] += velocity[i,k]
                    speedcollect[0,int(speedhisto[0,2])] = velocity[i,k]
                    speedhisto[0,2] += 1
            else:
                for j in range(1,steps2):
                    if value[i] > speedhisto[j-1,0] and value[i] <= speedhisto[j,0]:
                        if velocity[i,k] == velocity[i,k]:
                            speedhisto[j,1] += velocity[i,k]
                            speedcollect[j,int(speedhisto[j,2])] = velocity[i,k]
                            speedhisto[j,2] += 1
        speedhisto[:,1] /= speedhisto[:,2]
        for i in range(maxlength):
            for j in range(steps2):
                if speedcollect[j,i] == 0: speedcollect[j,i] = np.nan
        maxvelocity[k] = speedhisto[-1,1]
    
    if plothisto == True:
        x = particles[:,:,0]
        y = particles[:,:,1]
        factor = 1.1
        xmin = factor*np.nanmin(x)
        xmax = factor*np.nanmax(x)
        ymin = factor*np.nanmin(y)
        ymax = factor*np.nanmax(y)
        gridsizevalue = 15
        string_temp = savefolder + '/Plot2DHistogram/'
        if not os.path.exists(string_temp): os.makedirs(string_temp) 
        fig=plt.figure(1, figsize = (8.5,7) ) 
        #https://matplotlib.org/2.0.0/examples/pylab_examples/hexbin_demo.html
        ax = plt.subplot(1,1,1)
        hb = ax.hexbin(x, y, gridsize=gridsizevalue,mincnt=0.01)
        circle2 = plt.Circle((0, 0), boundaryradius*pxtomum, color='w', fill=False)
        ax.add_artist(circle2)
        cb = fig.colorbar(hb, ax=ax)
        cb.set_label('counts')
        ax.set_xlabel('x [µm]')
        ax.set_ylabel('y [µm]')
        ax.set_xlim(xmin,xmax)
        ax.set_ylim(ymin,ymax)
        ax.axis('equal')
        plt.tight_layout() 
        plt.savefig(string_temp+'Histogram'+str(plotnum)+'.pdf', format='pdf')
        plt.close(fig)#plt.show()     
    #-------------    
    if plotvelocityinout == True:
        smoothvalue = 5
        smoothit = np.zeros((maxlength,pnumber+1))
        for i in range(pnumber): smoothit[:,i] = smooth(velocity[:,i],smoothvalue)
        for i in range(maxlength): smoothit[i,-1] = np.mean(smoothit[i,:-1])
        vtotal = np.zeros((steps,pnumber))
        vin = np.zeros((steps,pnumber));vout = np.zeros((steps,pnumber))
        intensitydt = np.zeros((maxlength,pnumber))
        for p in range(pnumber): intensitydt[:,p] = np.gradient(particles[:,p,6])

        driftdelay = 1
        intensitystrength = np.zeros((steps,2))
        for i in range(steps): intensitystrength[i,0] = i/(steps-1)
        for i in range(1,steps): intensitystrength[i,1] = round((intensitystrength[i,0]+intensitystrength[i-1,0])/2,2)

        point = max(driftdelay,2)
        for p in range(pnumber):
            allintensitydata = np.zeros((maxlength,5))
            allintensitydata[:,1] = 2
            intensitystats = np.zeros((steps,6))
            intensitystats[:,0] = intensitystrength[:,1]
            count = 0
            inouttest = []
            for i in range(point,maxlength):
                if distancetocenter[i,p] < boundaryradius*pxtomum:
                    for j in range(1,steps):
                        if particles[i,p,6] < intensitystrength[j,0] and particles[i,p,6] > intensitystrength[j-1,0]:
                            allintensitydata[i,1] = intensitystrength[j,1]
                            allintensitydata[i,3] = smoothit[i,p]
                            if intensitydt[i,p] > 0:
                                intensitystats[j,1] += 1
                                intensitystats[j,2] += smoothit[i,p]
                                inouttest.append('inwards')
                                allintensitydata[i,2] = 0
                                count += 1
                            if intensitydt[i,p] < 0:
                                intensitystats[j,3] += 1
                                intensitystats[j,4] += smoothit[i,p]
                                inouttest.append('outwards')
                                allintensitydata[i,2] = 1
                                count += 1
                            j = steps
                if allintensitydata[i,1] == 2: allintensitydata[i,2] = 888
            intensitystats[:,5] = (intensitystats[:,2]+intensitystats[:,4])/(intensitystats[:,1]+intensitystats[:,3])
            intensitystats[:,2] /= intensitystats[:,1]
            intensitystats[:,4] /= intensitystats[:,3]
            vtotal[1:,p] = intensitystats[1:,5]
            for i in range(1,steps):
                vin[i,p] = round(intensitystats[i,2],2)
                vout[i,p] = round(intensitystats[i,4],2)

            alldata1 = list(zip(arange(point,maxlength),allintensitydata[point:,1],inouttest[point:],allintensitydata[point:,3]))
            alldata2 = pd.DataFrame(data = alldata1, columns=['step', 'distance', 'direction', 'velocity'])
            alldata3 = alldata2.replace(888, 'outside')
            
            string_temp = savefolder + '/PlotVelocityInandOut/'
            if not os.path.exists(string_temp): os.makedirs(string_temp)
            fig=plt.figure(1, figsize = (10,5) )
            ax4 = plt.subplot(1,1,1)
            ax4 = sns.violinplot(x="distance", y="velocity", hue="direction", hue_order = ['inwards','outwards'], order = intensitystrength[1:,1],
                                    data=alldata3, palette="coolwarm", split=True, scale="count", cut=0, inner=None)#,bw=.2, scale_hue=False
            ax4.set_ylabel('Velocity [µm/s]')
            ax4.set_xlabel('Sensed intensity')
            numwave=np.linspace(0,len(vin[1:]),len(vin[1:]),endpoint=False)
            for i in range(1,len(vin[1:,p])):
                ax4.plot([numwave[i-1],numwave[i-1]],[vin[i,p],vout[i,p]],color='k',linewidth=2.5)
            ax4.scatter(numwave,vin[1:,p],s=50,c='b')#,label = 'v in')
            ax4.scatter(numwave,vout[1:,p],s=50,c='r')#,label = 'v out')
            ax4.set_ylim([0,min(max(smoothit[:,p]),6)])
            ax4.set_xlim([steps-1.5,-0.5])
            #ax4.set_title('Particle '+str(p+1)+' - percentage data = '+str(round(count/maxlength*100,1))+'%',loc='left')
            ax4.set_title('Particle '+str(p+1)+' - delay = '+str(delay),loc='left')
            plt.legend(loc='upper left')
            plt.tight_layout() 
            plt.savefig(string_temp+'Velocity in and out - limited'+str(int(noise))+' - '+str(int(delay))+' - '+str(p+1)+'.pdf', format='pdf')
            plt.close(fig)#plt.show()
        
        string_temp = './DataIntensity/'                         
        if not os.path.exists(string_temp): os.makedirs(string_temp)                         #0                1         2        3          4          5          6          7            8            9
        np.savetxt(string_temp + 'DataIntensity'+str(filenumber)+'.txt', np.transpose([intensitystrength[1:,1],vin[1:,0],vin[1:,1],vout[1:,0],vout[1:,1],vtotal[1:,0],vtotal[1:,1]]),fmt='%1.6f')    
    #------------
    if plotcollision == True:
        connections = int(pnumber*(pnumber-1)/2)
        alldist = np.zeros((maxlength,connections))
        for i in range(maxlength):
            part1 = 0; part2 = 1
            for k in range(connections):
                x1=particles[i,part1,0];y1=particles[i,part1,1]
                x2=particles[i,part2,0];y2=particles[i,part2,1]
                dist=math.sqrt(pow(x1-x2,2)+pow(y1-y2,2))
                alldist[i,k] = dist
                part2 += 1
                if part2 == pnumber:
                    part1 += 1
                    part2 = part1 + 1
        close = 1.5*diameter
        timeclose = []
        part1 = 0; part2 = 1
        check = True
        for j in range(connections):
            count = 0
            for i in range(maxlength):
                if alldist[i,j] <= close:
                    check = False
                    count += 1
                else:
                    if check == False:
                        check = True
                        timeclose.append(count)
                        count = 0
            part2 += 1
            if part2 == pnumber:
                part1 += 1
                part2 = part1 + 1
        timeclose2 = np.zeros(len(timeclose))
        for i in range(len(timeclose)): timeclose2[i] = timeclose[i] * dt
        colltime = np.mean(timeclose2)

        bins = np.linspace(0.5,int(max(timeclose))+0.5,int(max(timeclose)+1))*dt
        string_temp = savefolder + '/PlotCollision/'
        if not os.path.exists(string_temp): os.makedirs(string_temp)
        fig=plt.figure(1, figsize = (7,5) )
        ax1 = plt.subplot(1,1,1)
        plt.hist(timeclose2,bins = bins)
        plt.xlabel('Time until particles seperate [s]')
        plt.ylabel('Counts')
        plt.savefig(string_temp+'Histogram collision time'+str(filenumber)+'.pdf', format='pdf')
        plt.close(fig)#plt.show()
        
    if plottrajectory == True:
        colorwave = cm.plasma(np.linspace(0, 1, pnumber))
        string_temp = savefolder + '/PlotTrajectory/'
        if not os.path.exists(string_temp): os.makedirs(string_temp)
        fig=plt.figure(1, figsize = (6,5) )
        ax = plt.subplot(1,1,1)
        ax.set_aspect('equal')
        for i in range(pnumber):
            ax.scatter(particles[starttraj,i,0],particles[starttraj,i,1],color='k',s = 100,zorder=3)
            ax.plot(particles[:,i,0],particles[:,i,1],color=colorwave[i],zorder=2)
            #ax.scatter(particles[:,i,4],particles[:,i,5],color=colorwave[i],s = 30,zorder=3)
            #ax.plot(save[:,i,0],save[:,i,1],zorder=1)
        circle2 = plt.Circle((0, 0), boundaryradius*pxtomum, color='k', fill=False)
        ax.add_artist(circle2)
        plt.xlabel('x-position [µm]')
        plt.ylabel('y-position [µm]')
        plt.tight_layout() 
        plt.savefig(string_temp+'Trajectory'+str(filenumber)+'.pdf', format='pdf')
        plt.close(fig)#plt.show()
    #---------------
    string_temp = './DataImportant/'                         
    if not os.path.exists(string_temp): os.makedirs(string_temp)                     #0    1    2      3              4                5               6      7                          
    np.savetxt(string_temp + 'Dataimportant'+str(filenumber)+'.txt', np.transpose([number,delay,noise,meandisttocom,meanintensity,percentagecluster,meansim,meanexp
    ,maxvelocity[0],maxvelocity[1],trajlength[0],trajlength[1],colltime]),fmt='%1.6f')

    print("File: ",int(filenumber),"  %s seconds " % round((time.time() - start_time),2),' - noise ',str(noise),' - delay ',str(delay))
    totaltime += round((time.time() - start_time),2)
print('Total time = ', totaltime)

--- 11.64 seconds ---


/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.object' is deprecated and will be removed in a future release. Use TdmsFile.properties to access properties of the root object, TdmsFile[group_name] to access a group object and TdmsFile[group_name][channel_name] to access a channel object.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.objects' is deprecated and will be removed in a future release. Use TdmsFile.groups() to access all groups in the file, and group.channels() to access all channels in a group.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:568: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:569: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:570: Runti

File:  12   23.88 seconds   - noise  14  - delay  21
--- 11.82 seconds ---


/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.object' is deprecated and will be removed in a future release. Use TdmsFile.properties to access properties of the root object, TdmsFile[group_name] to access a group object and TdmsFile[group_name][channel_name] to access a channel object.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.objects' is deprecated and will be removed in a future release. Use TdmsFile.groups() to access all groups in the file, and group.channels() to access all channels in a group.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:568: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:569: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:570: Runti

File:  13   23.51 seconds   - noise  17  - delay  1
--- 11.84 seconds ---


...

File:  22   23.33 seconds   - noise  26  - delay  13
--- 11.74 seconds ---


/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.object' is deprecated and will be removed in a future release. Use TdmsFile.properties to access properties of the root object, TdmsFile[group_name] to access a group object and TdmsFile[group_name][channel_name] to access a channel object.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.objects' is deprecated and will be removed in a future release. Use TdmsFile.groups() to access all groups in the file, and group.channels() to access all channels in a group.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:568: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:569: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:570: Runti

File:  23   23.25 seconds   - noise  26  - delay  17
--- 11.78 seconds ---


/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.object' is deprecated and will be removed in a future release. Use TdmsFile.properties to access properties of the root object, TdmsFile[group_name] to access a group object and TdmsFile[group_name][channel_name] to access a channel object.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/nptdms/tdms.py:1097: UserWarning: 'TdmsFile.objects' is deprecated and will be removed in a future release. Use TdmsFile.groups() to access all groups in the file, and group.channels() to access all channels in a group.
  warnings.warn(message)
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:568: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:569: RuntimeWarning: invalid value encountered in true_divide
/Users/alex/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:570: Runti

File:  24   23.1 seconds   - noise  26  - delay  21
Total time =  305.74000000000007
